In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [2]:
df= pd.read_csv("/kaggle/input/playground-series-s4e6/train.csv")
df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


Create the features(x) and the targets(y) to be used by the machine learning moddel from the provided data.

In [3]:
x = df.drop(["Target", "id"], axis=1)
y = df["Target"]

Have a look at the columns on your features dataframe

In [4]:
x.columns

Index(['Marital status', 'Application mode', 'Application order', 'Course',
       'Daytime/evening attendance', 'Previous qualification',
       'Previous qualification (grade)', 'Nacionality',
       'Mother's qualification', 'Father's qualification',
       'Mother's occupation', 'Father's occupation', 'Admission grade',
       'Displaced', 'Educational special needs', 'Debtor',
       'Tuition fees up to date', 'Gender', 'Scholarship holder',
       'Age at enrollment', 'International',
       'Curricular units 1st sem (credited)',
       'Curricular units 1st sem (enrolled)',
       'Curricular units 1st sem (evaluations)',
       'Curricular units 1st sem (approved)',
       'Curricular units 1st sem (grade)',
       'Curricular units 1st sem (without evaluations)',
       'Curricular units 2nd sem (credited)',
       'Curricular units 2nd sem (enrolled)',
       'Curricular units 2nd sem (evaluations)',
       'Curricular units 2nd sem (approved)',
       'Curricular units 2nd s

Import the necessary modules to help you with your feature engineering and to transform your data.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define the feature engineering function
def feature_engineering(x):
    if not isinstance(x, pd.DataFrame):
        train_df=pd.DataFrame(x)
    #if not isinstance(test_df, pd.DataFrame):
        #test_df = pd.DataFrame(test_df)
        
    # Categorical columns
    categorical_cols = [
        'Marital status', 'Application mode', 'Course', 'Daytime/evening attendance',
        'Previous qualification', 'Nacionality', 'Mother\'s qualification', 
        'Father\'s qualification', 'Mother\'s occupation', 'Father\'s occupation',
        'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date', 
        'Gender', 'Scholarship holder', 'International'
    ]
    
    # Numerical columns
    numerical_cols = [
        'Application order', 'Previous qualification (grade)', 'Admission grade', 
        'Age at enrollment', 'Curricular units 1st sem (credited)', 
        'Curricular units 1st sem (enrolled)', 'Curricular units 1st sem (evaluations)', 
        'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 
        'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (credited)', 
        'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 
        'Curricular units 2nd sem (approved)', 'Curricular units 2nd sem (grade)', 
        'Curricular units 2nd sem (without evaluations)', 'Unemployment rate', 
        'Inflation rate', 'GDP'
    ]
    
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Preprocessing for numerical data
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combining both preprocessing pipelines
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    
    return x, preprocessor

Split your data into training and testing sets.

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.1, random_state=42)


Use the function you created to transform your data, and fit the training data

In [7]:
x_train_processed, preprocessor = feature_engineering(x)

In [8]:
# Fit the transformer on the training data
preprocessor.fit(x_train)
    
# Transform both training and testing data
train_df_processed = preprocessor.transform(x_train)
test_df_processed = preprocessor.transform(x_test)

In [9]:
from sklearn.metrics import accuracy_score, roc_auc_score
#from lightgbm import LGBMClassifier
#model = LGBMClassifier()
#model.fit(x_train, y_train)
#accuracy_score(y_test, model.predict(x_test))

Create a simple machine learning model and train it on the training data. Using accuracy as the metrics, test the accuracy to see how it will perform in the real world.

In [10]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(x_train, y_train)
accuracy_score(y_test, model.predict(x_test))

0.8201896409020285

In [11]:
#import tensorflow as tf
#model2 = tf.keras.Sequential([tf.keras.layers.Dense(128, input_shape= None, activation= 'relu'),
           #tf.keras.layers.Dense(64, activation= 'relu'),
           #tf.keras.layers.Dense(32, activation= 'relu'),
           #tf.keras.layers.Dense(16, activation= 'relu'),
           #tf.keras.layers.Dropout(0.2),
           #tf.keras.layers.Flatten(),
           #tf.keras.layers.Dense(3, activation= 'softmax')
           #)]

In [12]:
#model2.compile(optimizer= 'adam', loss= 'sparse_categorical_crossentropy', metrics= ['accuracy'])"""

In [13]:
#model2.summary

In [14]:
#history= model.fit(x_train, y_train, epochs= 100)


In [15]:
def model_summary(model, x_test, y_test):
    print("Model Parameters:")
    print(model.get_params())
    print("\nFeature Importances:")
    print(model.feature_importances_)
   

In [16]:
model_summary(model, x_test, y_test)

Model Parameters:
{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}

Feature Importances:
[8.51789741e-04 2.97863888e-03 1.59955467e-03 7.60082081e-03
 9.65623711e-04 9.29337679e-04 2.42414235e-03 2.15101353e-04
 1.54715311e-03 2.12626051e-03 4.33157696e-03 2.93307384e-03
 4.50025756e-03 4.19357066e-04 1.39680703e-04 2.88524774e-03
 4.11243294e-02 1.84615330e-03 1.69712017e-02 8.26073678e-03
 2.10149340e-06 9.07316051e-04 7.03204318e-03 2.41675344e-02
 4.40816637e-02 1.14066942e-02 8.18789683e-04 1.64529844e-03
 1.75717294e-02 4.43380544e-02 6.95157756e-01 3.94995151e-02
 1.31330929e-03 2.81407196e-03 

In [17]:
test = pd.read_csv("/kaggle/input/playground-series-s4e6/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s4e6/sample_submission.csv")


In [18]:
test.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32


In [19]:

test,preprocessor = feature_engineering(test)
x_train = preprocessor.fit_transform(x)
x_test = preprocessor.transform(test)


In [20]:
model.fit(x_train, y)
prediction = model.predict(x_test)
submission["Target"] = prediction

submission.to_csv("submission.csv", index=False)

In [21]:
submission.head()

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled


In [22]:
submission.tail()

,id,Target
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout
51011,127529,Dropout


In [23]:
#saving my first ml model
# For a scikit-learn model
import joblib
joblib.dump(model, 'my_model.pkl')

['my_model.pkl']